# Getting Started with Melodia

Melodia is a library used for computing the differential geometry of the curves that describe the backbone of proteins. We designed it to be compatible with the Pandas (and BioPandas), BioPython and scikit-learn libraries. In this notebook, we will show some examples of its usage.

In [1]:
import os 
import dill

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from Bio.PDB import PDBParser, PDBIO

from sklearn.preprocessing import StandardScaler

In [2]:
# Import melodia library
import melodia as mel

## Creating a Pandas DataFrame object from a structure file

Melodia allows us to import a PDB file directly into a Pandas Dataframe. 

In [3]:
# Example with multiple chains
file_name = '2k5x.pdb'

In [4]:
# Example with multiple models
#file_name = '2lj5.pdb'

In [5]:
# The geometry_from_structure_file function is used to open a traditional PDB file and convert it
# to a Pandas DataFrame with its differential geometry

dfi = mel.geometry_from_structure_file(file_name)

In [6]:
dfi

## Model and Chain Selection

Due to the output data being in the Pandas format, we can use its notation for data processing. Thus, using the well-documented Pandas selection rules, model, and chain selection is trivial.

In [7]:
model = dfi['model'] == 0
chain = dfi['chain'] == 'A'

#dfo = dfi[model].copy() # Select only model
#dfo = dfi[chain].copy() # Select only chain
dfo = dfi[model & chain].copy() # Select both model and chain

In [8]:
dfo

## Basic Plots

### A Ramachandran Plot

In [9]:
cmap = sns.color_palette('Blues', as_cmap=True)
sns.jointplot(x='phi', y='psi', data=dfo, kind='kde', cmap=cmap, height=10, fill=True);

### A Differential Geometry based Ramachandran-Like Plot

One of the valuable applications of Melodia is a Ramachandran-like plot for the curvature and torsion values. It shows the general geometric behaviour of the protein backbone. 

In [10]:
sns.jointplot(x='curvature', y='torsion', data=dfo, kind='kde', cmap=cmap, height=10, fill=True);

### Apply scikit-learn Standard Scaler to the Geometric Attributes

For Machine and Deep Learning applications, it is essential to standardize the differential geometry values, as curvature and torsion present different scales. One possible procedure is the scikit-learn StandardScaler function. Here, we want to select only the glycine residues and visualize their geometric distribution.

In [11]:
# Define the autoscaler from scikit-learn
autoscaler = StandardScaler()

# Define features for scaling
features = ['curvature', 'torsion', 'arc_length', 'writhing']

# Scale the DataFrame
dfsd = dfo.copy()
dfsd[features] = autoscaler.fit_transform(dfsd[features])

In [12]:
name = dfsd['name'] == 'GLY'
sel_name = dfsd[name].copy()
sel_name

In [13]:
plot = sns.jointplot(x='curvature', y='torsion', data=sel_name, kind='kde', cmap=cmap, height=10, fill=True)
plot.fig.suptitle('GLY only')
plot.fig.subplots_adjust(top=0.95)

We can also select different residue types (GLY and GLU, for example), visualize their geometric distribution and compare their importance for the backbone shape.

In [14]:
# Select residues in a list of names
names = dfsd['name'].isin(['GLY', 'GLU'])
sel_names = dfsd[names].copy()
sel_names

In [15]:
plot = sns.jointplot(x='curvature', y='torsion', data=sel_names, hue='name', kind='kde', height=10)

***

Through this plot, it is easy to observe the different behaviours of the backbone geometry on these residues. The GLU residue shows a more comprehensive curvature range and tight torsion range, while GLY is the opposite, showing a limited curvature range and ample torsion range. Thus, GLU tends to curve the backbone in a plane, and GLY allows backbone twists to escape the plane. For a more thorough explanation of this behaviour, **see figure 16 in**:

Pitt, W.R., Montalvão, R.W. & Blundell, T.L. Polyphony: superposition independent methods for ensemble-based drug discovery. *BMC Bioinformatics* **15**, 324 (2014). https://doi.org/10.1186/1471-2105-15-324

***

In [16]:
plot = sns.jointplot(x='psi', y='phi', data=sel_names, hue='name', kind='kde', height=10)

***
The same is not valid for a simple Ramachandran Plot for the same dataset. Despite showing some differences between GLU and GLY residues, it is unclear what the differences imply for the protein backbone plasticity. Once one becomes experienced with the differential geometry of 3D curves, it is easy to interpret the backbone geometry.
***

## Creating geometric data from a BioPython parsed structure

Sometimes, it is helpful to compute the geometric properties directly from a BioPython data structure.  Melodia provides functions that can parse the geometry from a parsed structure for those situations.

In [17]:
parser = PDBParser()
name, ext = os.path.splitext(file_name)
structure = parser.get_structure(name, file_name)

### Create a Panda DataFrame from the parser

In [18]:
dfbio = mel.geometry_from_structure(structure)
dfbio

### Create a dictionary of attribute from the parser

Storing the geometric data in a flexible, lightweight structure is advantageous for general Python programming. Melodia provides a simple dictionary that stores a convenient data class. 

In [19]:
geo_dict = mel.geometry_dict_from_structure(structure)
geo_dict

In [20]:
# Access format 'model:chain'
geo_dict['0:A'].residues[0]

In [21]:
print(geo_dict['0:A'].residues[0].name)
print(geo_dict['0:A'].residues[0].curvature)
print(geo_dict['0:A'].residues[0].torsion)

## Data Storage

The Geometric Data in a data frame can be stored using **Pandas** functions. Additionally, the data in a dictionary can be stored in a file (pickled) using the **Dill library** (https://github.com/uqfoundation/dill).

In [22]:
dfbio.to_parquet('dfbio.parquet.gzip', compression='gzip')

In [23]:
dfbio_loaded = pd.read_parquet('dfbio.parquet.gzip')
dfbio_loaded

In [27]:
# Save dictionary to a file
with open('geo.dill', 'wb') as file:
    dill.dump(geo_dict, file)

In [28]:
# Load dictionary from a file
with open('geo.dill', 'rb') as file:
    geo_dict_loaded = dill.load(file)

In [29]:
geo_dict_loaded['0:A'].residues[0]

## Advanced Visualization

When using a BioPython structure object, Melodia can set its b-factor values as any geometric attribute. It permits us to visualize the value of the geometric descriptor as colours and radius project into the protein backbone.

We use the nglview widget for visualization (https://github.com/nglviewer/nglview).

In [30]:
mel.bfactor_from_geo(structure=structure, attribute='curvature', geo=geo_dict)
mel.bfactor_from_geo(structure=structure, attribute='torsion', geo=geo_dict)

### View structure as a putty model

In [31]:
mel.view_putty(structure[0], radius_scale=1.4, width=800, height=600)

### View structure as a cartoon

In [32]:
mel.view_cartoon(structure[0], width=800, height=600)

### View structure as a tube model

In [33]:
mel.view_tube(structure[0], width=800, height=600)

### Save the structure to a PDB file with the new bfactors



In [35]:
io = PDBIO()
io.set_structure(structure)
io.save('out.pdb')

After loading the PDB file in PyMol, the following command will create the correct visualization for the differential geometry:

1. show cartoon
2. cartoon putty
3. unset cartoon_smooth_loops
4. unset cartoon_flat_sheets
5. spectrum b

![PyMolImage](out.png)

## Access the Propensity Table for a target residue

Lastly, Melodia includes **Environmentally Constrained Amino Acid Propensity Tables**. These tables, divided into six *φ/ψ* areas, indicate the propensity for a residue to be replaced by another in a specific conformation. It is a very advanced topic discussed in the following articles:


1. Rinaldo W. Montalvão, Richard E. Smith, Simon C. Lovell, Tom L. Blundell, CHORAL: a differential geometry approach to the prediction of the cores of protein structures, *Bioinformatics*, Volume 21, Issue 19, January 2005, Pages 3719–3725, https://doi.org/10.1093/bioinformatics/bti595

2. Deane CM, Blundell TL. CODA: a combined algorithm for predicting the structurally variable regions of protein models. *Protein Sci.* 2001 Mar;10(3):599-612. https://doi.org/10.1110/ps.37601

In [32]:
ptable = mel.PropensityTable()

In [33]:
phi = -82.0
psi =  55.0
ptable.get_score(target='F', residue='A', phi=phi, psi=psi)